A notebook for capturing MTF chart image data for each channel of OROS (the OROCHI Simulator).

# Process Overview

- Find Exposure time - aim for 3000 DN exposure, for SNR of ~120.
- Set target in position
- Capture 25 repeat images, save the average and standard deviation
- Capture dark images at the given exposures.

# Setup

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import orochi_sim_ctrl as osc

In [ ]:
ic = osc.start_ic()
camera_config = osc.load_camera_config()
cameras = osc.connect_cameras(ic, camera_config)
osc.configure_cameras(cameras)

# Find Exposures

In [ ]:
mtf_exposures = osc.find_channel_exposures(cameras, init_t_exp=0.4, target=0.73, n_hot=800,tol=10, limit=10, roi=True)

# Capture Illuminated Images

In [ ]:
subject = 'mtf_9_illum_27062023'
check_stk = {}
for camera in cameras:
    t_exp = mtf_exposures[camera.name]
    camera.set_exposure(t_exp)
    ave, std = camera.image_capture_repeat(n=25, roi=True)
    print(ave.shape)
    check_stk[camera.name] = ave
    name = f'mtf_1_{camera.name}_ave'
    camera.save_image(name, subject, 'ave', ave)
    name = f'mtf_1_{camera.name}_std'
    camera.save_image(name, subject, 'std', std)    

In [ ]:
for camera in cameras:
    img = check_stk[camera.name]
    camera.show_image(img, camera.name)

# Capture Dark Images

In [ ]:
subject = 'mtf_9_dark_27062023'
for camera in cameras:
    t_exp = mtf_exposures[camera.name]
    camera.set_exposure(t_exp)
    ave, std = camera.image_capture_repeat(n=25)
    name = f'dark_{camera.name}_ave'
    camera.save_image(name, subject, 'ave', ave)
    name = f'dark_{camera.name}_std'
    camera.save_image(name, subject, 'std', std)    